<a href="https://colab.research.google.com/github/adamstra/MyModel/blob/main/VGG-16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# prompt: unzipper ce ficher le path: /content/drive/MyDrive/MyData.zip

import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/MyData.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/data')

In [16]:
import os
import cv2
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [17]:
def load_annotations(annotation_path):
    """Charge les annotations à partir d'un fichier XML Pascal VOC."""
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    boxes = []
    for obj in root.iter('object'):
        bndbox = obj.find('bndbox')
        box = [
            int(bndbox.find('xmin').text),
            int(bndbox.find('ymin').text),
            int(bndbox.find('xmax').text),
            int(bndbox.find('ymax').text)
        ]
        boxes.append(box)
    return np.array(boxes)

In [18]:
def load_data(image_dir):
    """Charge les images et leurs annotations."""
    images = []
    labels = []
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(image_dir, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Redimensionner les images pour qu'elles soient compatibles avec VGG16
            xml_path = img_path.replace('.jpg', '.xml')
            boxes = load_annotations(xml_path)  # Charger les annotations
            images.append(img)
            labels.append(boxes)

    return np.array(images), labels

In [19]:
# 2. Charger et séparer les données en train/validation (80% / 20%)
image_dir = '/content/drive/MyDrive/data'
images, labels = load_data(image_dir)

# 3. Fixer une limite de nombre de bounding boxes par image (par ex: 5 max)
max_boxes = 10
def pad_bounding_boxes(boxes):
    """Ajoute du padding si une image a moins de 'max_boxes' bounding boxes.
    Tronque si une image a plus de 'max_boxes' bounding boxes."""
    padded_boxes = np.zeros((max_boxes, 4))  # Créer un tableau vide pour max_boxes bounding boxes
    num_boxes = min(len(boxes), max_boxes)  # Prendre le minimum entre le nombre de boxes et max_boxes
    padded_boxes[:num_boxes] = boxes[:num_boxes]  # Remplir avec les vraies bounding boxes, tronquées si nécessaire
    return padded_boxes

padded_labels = [pad_bounding_boxes(box) for box in labels]

In [20]:
# 4. Séparer en train et validation
train_images, val_images, train_labels, val_labels = train_test_split(images, padded_labels, test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [13]:
# 5. Créer un dataset TensorFlow avec padding pour les bounding boxes
def process_data(images, labels):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    return dataset.map(lambda x, y: (tf.image.convert_image_dtype(x, tf.float32), y))

In [14]:
# 6. Créer des datasets train/validation
train_dataset = process_data(train_images, train_labels).batch(32)
val_dataset = process_data(val_images, val_labels).batch(32)

NameError: name 'train_images' is not defined

In [20]:
# 7. Définir et construire le modèle VGG16 pour la détection d'objets
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches du modèle de base
for layer in base_model.layers:
    layer.trainable = False

In [29]:
# Ajouter des couches fully-connected
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
# Assuming your target labels are flattened (40 values for 10 objects)
# Reshape the output layer to match the target shape [?, 40] for 10 objects with 4 coordinates
output = Dense(40, activation='linear')(x)
output = Reshape((10, 4))(output)  # Reshape to (10, 4)

model = Model(inputs=base_model.input, outputs=output)

In [30]:
# 8. Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [31]:
# 9. Ajouter un callback pour sauvegarder les checkpoints du meilleur modèle
checkpoint_filepath = '/kaggle/working/checkpoints/vgg16_checkpoint.keras' # Change the file extension to .keras
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

In [ ]:
# 10. Entraîner le modèle avec validation et sauvegarde des checkpoints
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    steps_per_epoch=len(train_images) // 32,
    validation_steps=len(val_images) // 32,
    callbacks=[checkpoint_callback]
)

Epoch 1/10
 1/26 ━━━━━━━━━━━━━━━━━━━━ 9:32 23s/step - accuracy: 0.2313 - loss: 70385.8672